# Final Project
### Import precomp libs

In [1]:
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
%load_ext line_profiler

In [1]:
#import the spacy english model
# "python -m spacy download en"   to download english models

import cPickle as pickle 
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import coo_matrix, hstack

In [17]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape

(404290, 6) (2345796, 3)


In [18]:
train

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [4]:
pd.isnull(test.loc[379205]['question2'])

True

In [5]:
from nltk.stem.porter import PorterStemmer
def stem(l):
    porter = PorterStemmer()
    for w in l:
        porter.stem(w)
stem('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

In [6]:
def parse(sentence):
    bow = []
    sentence = sentence.decode('utf-8')
    p = nlp(sentence)
    for word in p:
        if word.is_stop: continue
        bow.append(word.lemma_)
    return Counter(bow)
        
parse('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

Counter({u'(': 1,
         u')': 1,
         u'-': 2,
         u'?': 1,
         u'buy': 1,
         u'diamond': 1,
         u'koh': 1,
         u'kohinoor': 1,
         u'man': 1,
         u'noor': 1,
         u'shirt': 1,
         u'store': 1,
         u'story': 1,
         u'walk': 1})

In [8]:
def make_vecs(corpus, results=False):
    vecs = []
    results = []
    n = len(corpus)
    for i in range(n): 
        if i % 10000 == 0: print i
        f = corpus.loc[i]
        q1, q2 = f['question1'], f['question2']
        try: 
            p1 = parse(q1)
            p2 = parse(q2)
            diff1 = p1 - p2
            diff2 = p2 - p1
            vecs.append((diff1, diff2))
            if results: results.append(f['is_duplicate'])
        except Exception as e:
            vecs.append('')
            print(i, e)
    return vecs, results

In [ ]:
# fv_train, target = make_vecs(train, results=True)

In [ ]:
# fv_test, results = make_vecs(test)

In [9]:
# with open('cfv_train','w') as f:
#     pickle.dump(fv_train_,f)
# with open('ctarget','w') as f:
#     pickle.dump(target,f)
# # with open('fv_test','w') as f:
#     pickle.dump(fv_test,f)

NameError: name 'fv_train_' is not defined

## load from pickle

In [3]:
fv_train = pickle.load(open('cfv_train', 'rb'))
fv_test = pickle.load(open('cfv_test', 'rb'))
target = pickle.load(open('ctarget', 'rb'))

In [11]:
print(fv_train[10])

(Counter({u'slit': 1, u'separation': 1, u'biprism': 1, u'fresnel': 1, u'method': 1, u'find': 1}), Counter({u'laptops': 1, u'thing': 1, u'reliability': 1, u'durability': 1, u'technician': 1, u'component': 1, u'tell': 1}))


In [50]:
v1, v2 = zip(*fv_train_p)

In [52]:
v1 = v1[:1000]

In [4]:
def vectorize(vectors, vectorizer=None):
    for i, v in enumerate(vectors):
        if v == '': vectors[i] = (Counter(), Counter())
    v1, v2 = zip(*vectors)
    if not vectorizer:
        vectorizer = DictVectorizer().fit(v1 + v2)
    x1 = vectorizer.transform(v1)
    x2 = vectorizer.transform(v2)
    X = hstack([x1,x2])
    return X, vectorizer

In [6]:
X, vectorizer = vectorize(fv_train)

#### Validation

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
n_estimators = 10
clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel='linear'), max_samples=1.0 / n_estimators, n_estimators=n_estimators, n_jobs = -1))
scores = cross_val_score(clf, X, target, cv=3)
print "Log Loss: %0.5f" % scores.mean()

In [8]:
n_estimators = 20
clfs = [MultinomialNB(), SGDClassifier(loss='log'), LogisticRegression()]

for clf in clfs:
    print clf
    scores = cross_val_score(clf, X, target, cv=3, scoring='neg_log_loss')
    print "Log Loss: %0.5f" % scores.mean()

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Log Loss: -0.60527
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
Log Loss: -0.52845
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Log Loss: -0.47354


### Testing

In [28]:
f = open('submission', 'w')
f.write('test_id,is_duplicate\n')

In [11]:
print fv_test[1]

(Counter({u'24': 1, u'age': 1, u'?': 1}), Counter({u'require': 1}))


In [20]:
X_test,_ = vectorize(fv_test, vectorizer)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [14]:
clf = LogisticRegression().fit(X, target)

In [22]:
prob = clf.predict_proba(fv_test[0])

In [30]:
for i, p in enumerate(prob):
    f.write(str(i) + ',' + str(p[0]) + '\n')